## Data Cleaning HW
## Miles Voenell 11/15/25


Raw Data: 

In [ ]:
import pandas as pd

#original file
df = pd.read_csv("stock_market.csv")
#shape
print("Shape of dataset:", df.shape)
#head
print(df.head())
#schema/null summary
print(df.dtypes)
print(df.isnull().sum())

Shape of dataset: (200, 10)
   Trade Date Ticker Open Price Close Price    Volume  \
0  09/01/2025  AAPL      192.47      191.90  21646536   
1  09/01/2025   MSFT     456.51      459.76       NaN   
2  09/01/2025    NaN     151.27      150.45  20405836   
3  09/01/2025      -     139.61      141.17  12795579   
4  09/01/2025      -    1190.21     1198.56  16389274   

                    Sector Validated Currency  Exchange       Notes  
0              Technology         na        -    NASDAQ     gap up   
1               Technology         -       na        na         NaN  
2  Consumer Discretionary         na      usd   NASDAQ            -  
3                        -       Yes      NaN      NYSE   gap down   
4          Semiconductors         na      USD      NYSE         NaN  
Trade Date     object
Ticker         object
Open Price     object
Close Price    object
Volume         object
Sector         object
Validated      object
Currency       object
Exchange       object
Notes      

In [31]:
import pandas as pd
import numpy as np

# load csv
df = pd.read_csv("stock_market.csv", dtype=str)

#remove dash and underscore
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

#remove white spaces
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#find and replace all possibilies for null/empty
null_like = ["", " ", "\t", "\n", "-", "—", "NA", "N/A", "na", "n/a", "None", "none", "Null", "null"]
df = df.replace(null_like, np.nan)
df = df.replace(r'^\s*$', np.nan, regex=True)

#check/remove duplicates
df = df.drop_duplicates()

#find and fill in boolean values for "Validated"
yes_vals = {"y", "yes", "true", "t", "1", "yep"}
no_vals  = {"n", "no", "false", "f", "0"}

def to_boolean(val):
    if isinstance(val, str):
        v = val.strip().lower()
        if v in yes_vals:
            return True
        if v in no_vals:
            return False
    return val

df = df.applymap(to_boolean)

if "validated" in df.columns:
    df["validated"] = df["validated"].astype("boolean")

#force date into cleaner format
if "trade_date" in df.columns:
    df["trade_date"] = pd.to_datetime(df["trade_date"], errors="coerce").dt.strftime("%Y-%m-%d")

#fix capitalization
text_cols = ["ticker", "sector", "currency", "exchange", "notes"]
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].where(df[col].notna(), np.nan)
        df[col] = df[col].str.title()

#numeric values
numeric_cols = ["open_price", "close_price", "volume"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

#save as parquet
df.to_parquet("cleaned.parquet", index=False)
print("✅ Cleaned CSV saved as cleaned.parquet")


CSV saved as cleaned.parquet


C:\Users\mvoen\AppData\Local\Temp\ipykernel_8404\2516659592.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\mvoen\AppData\Local\Temp\ipykernel_8404\2516659592.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(to_boolean)


In [32]:
import pandas as pd

#load parquet file
df_cleaned = pd.read_parquet("cleaned.parquet")
print(df_cleaned.head(10))

#shape
print("Shape:", df_cleaned.shape)

#see if the data types are correct
print(df_cleaned.dtypes)

#check missing
print(df_cleaned.isna().sum())

   trade_date ticker  open_price  close_price      volume  \
0  2025-09-01   Aapl      192.47       191.90  21646536.0   
1  2025-09-01   Msft      456.51       459.76         NaN   
2  2025-09-01   None      151.27       150.45  20405836.0   
3  2025-09-01   None      139.61       141.17  12795579.0   
4  2025-09-01   None     1190.21      1198.56  16389274.0   
5  2025-09-01   Tsla      245.81       244.27  13350388.0   
6  2025-09-01   Meta      493.91       491.42  12963843.0   
7  2025-09-01   Nflx      603.91       599.53         NaN   
8  2025-09-02   Aapl         NaN       188.92  10513474.0   
9  2025-09-02   Msft      450.93          NaN  17277392.0   

                   sector  validated currency exchange         notes  
0              Technology       <NA>     None   Nasdaq        Gap Up  
1              Technology       <NA>     None     None          None  
2  Consumer Discretionary       <NA>      Usd   Nasdaq          None  
3                    None       True     Non

In [ ]:
import pandas as pd

df = pd.read_parquet("cleaned.parquet")
df['trade_date'] = pd.to_datetime(df['trade_date'], errors='coerce')

#weekly total volume per sector
weekly_volume_sector = (
    df.groupby([pd.Grouper(key='trade_date', freq='W-MON'), 'sector'], dropna=False)['volume']
      .sum(min_count=1)
      .reset_index()
      .sort_values(['trade_date', 'sector'])
)
print(weekly_volume_sector.head(), "\n")
weekly_volume_sector.to_parquet("agg1.parquet", index=False)

#weekly average % return per sector

df['daily_return'] = (df['close_price'] - df['open_price']) / df['open_price']

weekly_avg_return_sector = (
    df.groupby([pd.Grouper(key='trade_date', freq='W-MON'), 'sector'], dropna=False)['daily_return']
      .mean()
      .reset_index()
      .sort_values(['trade_date', 'sector'])
)
print(weekly_avg_return_sector.head(), "\n")
weekly_avg_return_sector.to_parquet("agg2.parquet", index=False)

#weekly count of validated trades per sector

weekly_validated_count = (
    df[df['validated'] == True]
      .groupby([pd.Grouper(key='trade_date', freq='W-MON'), 'sector'], dropna=False)['validated']
      .count()
      .reset_index()
      .rename(columns={'validated': 'validated_count'})
      .sort_values(['trade_date', 'sector'])
)
print(weekly_validated_count.head())
weekly_validated_count.to_parquet("agg3.parquet", index=False)


  trade_date                  sector      volume
0 2025-09-01              Automotive  13350388.0
1 2025-09-01  Communication Services  12963843.0
2 2025-09-01  Consumer Discretionary  20405836.0
3 2025-09-01          Semiconductors  16389274.0
4 2025-09-01              Technology  21646536.0 

  trade_date                  sector  daily_return
0 2025-09-01              Automotive     -0.006265
1 2025-09-01  Communication Services     -0.006147
2 2025-09-01  Consumer Discretionary     -0.005421
3 2025-09-01          Semiconductors      0.007016
4 2025-09-01              Technology      0.002079 

  trade_date                  sector  validated_count
0 2025-09-01                     NaN                1
1 2025-09-08              Automotive                2
2 2025-09-08  Communication Services                6
3 2025-09-08  Consumer Discretionary                2
4 2025-09-08          Semiconductors                2
